# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key
api_key = g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
cities = pd.read_csv("output_data/WeatherPy_df.csv")
cities_df = pd.DataFrame(cities)
cities_df.head()

,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Datetime
0,ushuaia,-54.80,-68.30,35.60,80,75,25.28,AR,1603190951
1,pakxe,15.12,105.78,80.60,88,90,6.93,LA,1603191358
2,hermanus,-34.42,19.23,61.00,67,15,5.99,ZA,1603190939
3,aksu,41.12,80.26,66.99,23,0,4.79,CN,1603191333
4,carnarvon,-24.87,113.63,71.60,68,0,19.46,AU,1603191224


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
#Configure gmaps
gmaps.configure(api_key=g_key)

# Store lat amd lng in loacxtions
locations = cities_df[["Latitude", "Longitude"]]
humidity = cities_df["Humidity"]

In [14]:
# Plot heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add lsyer
fig.add_layer(heat_layer)

## Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
max_intensity

100

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [55]:
# Narrow cities to those with max temp between 60 and 70 and cloudiness equal to 0
narrowed_city_df = cities_df.loc[(cities["Max_Temp"] > 60) & (cities_df["Max_Temp"]  < 70) & (cities_df["Cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how="any")
#narrowed_city_df.reset_index(inplace=True)
#del narrowed_city_df["Index"]
narrowed_city_df.head()

,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Datetime
3,aksu,41.12,80.26,66.99,23,0,4.79,CN,1603191333
33,cidreira,-30.18,-50.21,69.60,73,0,4.99,BR,1603191249
66,kashi,39.45,75.98,68.00,11,0,4.47,CN,1603191372
77,saldanha,-33.01,17.94,68.00,45,0,8.05,ZA,1603191266
83,stratonion,40.51,23.82,69.01,46,0,5.95,GR,1603191376


In [56]:
narrowed_city_df.shape

(10, 9)

In [63]:
len(narrowed_city_df)

10

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [69]:
hotel_df =  narrowed_city_df.loc[:,["City", "Country", "Latitude", "Longitude"]]

# Add "Hotel Name" column to DatasFrame
hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Latitude,Longitude,Hotel Name
3,aksu,CN,41.12,80.26,
33,cidreira,BR,-30.18,-50.21,
66,kashi,CN,39.45,75.98,
77,saldanha,ZA,-33.01,17.94,
83,stratonion,GR,40.51,23.82,
308,atbasar,KZ,51.80,68.33,
395,hirado,JP,33.36,129.55,
426,kalat,PK,29.02,66.59,
446,sonoita,MX,31.85,-112.83,
467,fukue,JP,32.69,128.84,


In [79]:
base_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type": "hotel",
         "keyword": "hotel",
         "radius": 5000,
         "key": g_key}

for index, row in hotel_df.iterrows():
    #get city, coordinates(lat&lng), Int froim hotel_df
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
    
    # add keyword to params dictg
    params["location"] = f"{lat},{lng}"
    
    # construct url and amke APi request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    #extract results
    results = response['results']
    
    # output hotel nsme to hotel_df
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    
    # if no avaialable hotels, show missing field
    except (KeyError, IndexError):
        print(f"Missing field/result...skipping.")
        
        print("-----------")
    
        
# print end of search after searching is complete
print("-----End of Search-----")       
    

Retrieving Results for Index 3: aksu.
Closest hotel in aksu is Pudong Holiday Hotel.
Retrieving Results for Index 33: cidreira.
Closest hotel in cidreira is Hotel Castelo.
Retrieving Results for Index 66: kashi.
Closest hotel in kashi is Radisson Blu Hotel Kashgar.
Retrieving Results for Index 77: saldanha.
Closest hotel in saldanha is Protea Hotel by Marriott Saldanha Bay.
Retrieving Results for Index 83: stratonion.
Missing field/result...skipping.
-----------
Retrieving Results for Index 308: atbasar.
Closest hotel in atbasar is The Ritz-Carlton, Astana.
Retrieving Results for Index 395: hirado.
Closest hotel in hirado is Yukai Resort Hotel Ranpu.
Retrieving Results for Index 426: kalat.
Closest hotel in kalat is Balochistan Hotel.
Retrieving Results for Index 446: sonoita.
Closest hotel in sonoita is Hotel Excelsior.
Retrieving Results for Index 467: fukue.
Closest hotel in fukue is Hotel Sou.
-----End of Search-----


In [80]:
# Display hotel_df
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
3,aksu,CN,41.12,80.26,Pudong Holiday Hotel
33,cidreira,BR,-30.18,-50.21,Hotel Castelo
66,kashi,CN,39.45,75.98,Radisson Blu Hotel Kashgar
77,saldanha,ZA,-33.01,17.94,Protea Hotel by Marriott Saldanha Bay
83,stratonion,GR,40.51,23.82,
308,atbasar,KZ,51.80,68.33,"The Ritz-Carlton, Astana"
395,hirado,JP,33.36,129.55,Yukai Resort Hotel Ranpu
426,kalat,PK,29.02,66.59,Balochistan Hotel
446,sonoita,MX,31.85,-112.83,Hotel Excelsior
467,fukue,JP,32.69,128.84,Hotel Sou


In [83]:
hotel_df.count()

City          10
Country       10
Latitude      10
Longitude     10
Hotel Name    10
dtype: int64

In [84]:
hotel_df.isnull()

,City,Country,Latitude,Longitude,Hotel Name
3,False,False,False,False,False
33,False,False,False,False,False
66,False,False,False,False,False
77,False,False,False,False,False
83,False,False,False,False,False
308,False,False,False,False,False
395,False,False,False,False,False
426,False,False,False,False,False
446,False,False,False,False,False
467,False,False,False,False,False


In [86]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
#hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
#locations = hotel_df[["Lat", "Lng"]]
locations = hotel_df[["Latitude", "Longitude"]]

In [89]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# add the layer to the map
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))